## US Core Examples Transaction Bundle Maker
### bundle all resources in folder as a transaction using PUT interactions

- python version 3.6+
- get data from folder
- add meta tag "2022-05-US-Core"
- bundle
- update and resolve ids
- validate and save

In [63]:
from fhir.resources import construct_fhir_element
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
#from fhirclient.r4models import bundle as B
#import fhirclient.r4models.fhirdate as D
import os, uuid
from datetime import datetime

bundle_id = '2022-05-US-Core-upload'
bundle_type = 'transaction'
base_url = 'http://hapi.fhir.org/baseR4'

###  write to file

In [64]:
def write_file(r): # write file
    
    #out_path = '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/'  # append forward slash
    out_path ='r4'
    print(f'writing to ... {out_path}/Bundle-{bundle_id}.json') 
    with open(f'{out_path}/Bundle-{bundle_id}.json', 'w') as f:
        f.write(r)


### Validate Resource

In [65]:
def validate(resource_type,data): #as json data
 
    #fhir_test_server = 'http://test.fhir.org/r4'
    fhir_test_server = 'http://hapi.fhir.org/baseR4'
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/{resource_type}/$validate', params = params, headers = headers, data = data)
    display(Markdown(f'# {r.status_code} {r.reason}\n\n{r.json()["text"]["div"]}'))
    # view  output
    # return (r.json()["text"]["div"])

### open json file and return as dict

In [66]:
def open_file(f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))

### Get files in current path

In [67]:
in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package/example' #'r4'

files = [x for x in os.listdir(in_path) if x.endswith(".json") and x not in [".index.json"]]
for i,f in enumerate(sorted(files)):
    print(f"{i+1}) {f}")


1) AllergyIntolerance-example.json
2) Bundle-66c8856b-ba11-4876-8aa8-467aad8c11a2.json
3) Bundle-c887e62f-6166-419f-8268-b5ecd6c7b901.json
4) Bundle-uscore-mo3.json
5) CarePlan-colonoscopy.json
6) CareTeam-example.json
7) Condition-condition-SDOH-example.json
8) Condition-encounter-diagnosis-example1.json
9) Condition-encounter-diagnosis-example2.json
10) Condition-health-concern-example.json
11) Device-udi-1.json
12) Device-udi-2.json
13) Device-udi-3.json
14) DiagnosticReport-bone-density-report.json
15) DiagnosticReport-cardiology-report.json
16) DiagnosticReport-cbc.json
17) DiagnosticReport-chest-xray-report.json
18) DiagnosticReport-metabolic-panel.json
19) DocumentReference-episode-summary.json
20) Encounter-1036.json
21) Encounter-delivery.json
22) Encounter-example-1.json
23) Goal-goal-1.json
24) Goal-goal-sdoh-2.json
25) Immunization-imm-1.json
26) Location-hl7east.json
27) Location-hospital.json
28) Media-chest-xray.json
29) Media-ekg-strip.json
30) Medication-uscore-med1.js

### create Bundle 'b'  change the id for unique Bundles!!!

In [68]:
#b = B.Bundle()

b_dict =dict(
id =  bundle_id,
type = bundle_type,
timestamp = datetime.utcnow(),
)

b = construct_fhir_element('Bundle',b_dict
)

#b.as_json()
b, b.json()    

(Bundle(resource_type='Bundle', fhir_comments=None, id='2022-05-US-Core-upload', id__ext=None, implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, entry=None, identifier=None, link=None, signature=None, timestamp=datetime.datetime(2022, 4, 29, 15, 14, 7, 163453), timestamp__ext=None, total=None, total__ext=None, type='transaction', type__ext=None),
 '{"resourceType":"Bundle","id":"2022-05-US-Core-upload","type":"transaction","timestamp":"2022-04-29T15:14:07.163453"}')

### Add resources to bundle

 - add tag to each resource first
 - remove text elements

In [69]:
def get_recursively(search_dict, field):
    """
    Takes a dict with nested lists and dicts,
    and searches all dicts for a key of the field
    provided.
    """
    fields_found = []

    for key, value in search_dict.items():

        if key == field:
            fields_found.append(value)

        elif isinstance(value, dict):
            results = get_recursively(value, field)
            for result in results:
                fields_found.append(result)

        elif isinstance(value, list):
            #print(f'{value} is list')
            for item in value:
                if isinstance(item, dict):
                    more_results = get_recursively(item, field)
                    for another_result in more_results:
                        fields_found.append(another_result)

    return fields_found

In [75]:
from fhir.resources import 
b.entry = []
type_list = []
ref_map = {}

for i in files:
    r = open_file(i)
    if r["resourceType"] in ["Bundle"]: # no bundles for now!
        continue
    
    #update ids to new unique ids for upload
    type_list.append(r["resourceType"])
    old_ref = f'{r["resourceType"]}/{r["id"]}'
    new_id = f'202201uscore-{r["resourceType"].lower()}-example-{type_list.count(r["resourceType"])}'
    new_ref = f'{r["resourceType"]}/{new_id}'
    #print(old_ref, new_ref)
    ref_map[old_ref] = new_ref

    r.pop('text', None) # remove text elements from Bundle
    r['meta']['tag']= [{"code":"202201uscore",}] # add meta tag
    r["id"] = new_id
    
    e = dict(
        resource = r,
        fullUrl = f'{base_url}/{new_ref}'
        )

    if bundle_type in ['transaction', 'batch']:
        e['request'] = dict(
                    method = 'PUT',
                    url = new_ref,
                    )
    print(new_ref)
    try:
        be = construct_fhir_element('BundleEntry',e)
    except ValidationError as e:
        print(e.message)
        
    b.entry.append(be)
    b.json()

Observation/202201uscore-observation-example-1
Observation/202201uscore-observation-example-2
RelatedPerson/202201uscore-relatedperson-example-1
Encounter/202201uscore-encounter-example-1
Observation/202201uscore-observation-example-3
MedicationRequest/202201uscore-medicationrequest-example-1
ServiceRequest/202201uscore-servicerequest-example-1
Observation/202201uscore-observation-example-4
Observation/202201uscore-observation-example-5
Observation/202201uscore-observation-example-6
Observation/202201uscore-observation-example-7
Observation/202201uscore-observation-example-8
Observation/202201uscore-observation-example-9
Observation/202201uscore-observation-example-10
Observation/202201uscore-observation-example-11
Observation/202201uscore-observation-example-12
Observation/202201uscore-observation-example-13
Organization/202201uscore-organization-example-1
QuestionnaireResponse/202201uscore-questionnaireresponse-example-1
Observation/202201uscore-observation-example-14
Observation/202

NameError: name 'ValidationError' is not defined


### Resort bundle for transaction
 - resort for transaction to make sure all the referred to resources are before the referring ones
 - rename ids for PUT to server


In [ ]:

transaction_order = dict(
Patient = 0,
Organization = 1,
Practitioner = 2,
Location = 4,
PractitionerRole = 5,
Encounter = 6,
Observation = 7,
Media = 8,
Medication = 9,
RelatedPerson = 10,
)

b.entry.sort(key=lambda x: transaction_order.get(x.resource.resource_type,11))
# for i,r in enumerate(b.entry):
#     print(f'{i+1})',r.resource.resource_type)

# change all the references to the new ids
b_json = dumps(b.as_json(),indent=3,)
for old_ref, new_ref in ref_map.items():
     b_json = b_json.replace(old_ref,new_ref)  
# display
print(f'{b_json[0:1000]}\n...\n{b_json[-1000:]}')

### Validate as JSON String

In [ ]:
data = b_json
resource_type = b.resource_type

validate(resource_type,data)

### Write to file

In [ ]:
write_file(b_json)

### print out FHIR_id Mapping table

In [ ]:
my_table = '|US Core IG FHIR ID|US Core Connectathon 29 FHIR ID|\n|---|---|'
for k,v in ref_map.items():
    my_table = (f'{my_table}\n| {k} | {v} |')
display(Markdown(my_table))
print(my_table)